In [1]:
import nannyml as nml
from IPython.display import display

reference_df, analysis_df, analysis_targets_df = nml.load_synthetic_car_loan_dataset()
analysis_full_df = analysis_df.merge(analysis_targets_df, left_index=True, right_index=True)

column_names = [
    'car_value', 'salary_range', 'debt_to_income_ratio', 'loan_length', 'repaid_loan_on_prev_car', 'size_of_downpayment', 'driver_tenure', 'y_pred_proba', 'y_pred', 'repaid'
]

univ_calc = nml.UnivariateDriftCalculator(
    column_names=column_names,
    treat_as_categorical=['y_pred', 'repaid'],
    timestamp_column_name='timestamp',
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon'],
    categorical_methods=['chi2', 'jensen_shannon'],
    chunk_size=5000
)

univ_calc.fit(reference_df)
univariate_results = univ_calc.calculate(analysis_full_df)
display(univariate_results.filter(period='analysis', column_names=['debt_to_income_ratio']).to_df())

alert_count_ranker = nml.AlertCountRanker()
alert_count_ranked_features = alert_count_ranker.rank(
    univariate_results.filter(methods=['jensen_shannon']),
    only_drifting = False)
display(alert_count_ranked_features)

estimated_calc = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    metrics=['roc_auc', 'recall'],
    chunk_size=5000,
    problem_type='classification_binary',
)
estimated_calc.fit(reference_df)
estimated_perf_results = estimated_calc.estimate(analysis_full_df)
display(estimated_perf_results.filter(period='analysis').to_df())

realized_calc = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    problem_type='classification_binary',
    metrics=['roc_auc', 'recall',],
    chunk_size=5000)
realized_calc.fit(reference_df)
realized_perf_results = realized_calc.calculate(analysis_full_df)
display(realized_perf_results.filter(period='analysis').to_df())

ranker1 = nml.CorrelationRanker()

# ranker fits on one metric and reference period data only
ranker1.fit(
    estimated_perf_results.filter(period='reference', metrics=['roc_auc']))
# ranker ranks on one drift method and one performance metric
correlation_ranked_features1 = ranker1.rank(
    univariate_results.filter(methods=['jensen_shannon']),
    estimated_perf_results.filter(metrics=['roc_auc']),
    only_drifting = False)

display(correlation_ranked_features1)

ranker2 = nml.CorrelationRanker()

# ranker fits on one metric and reference period data only
ranker2.fit(
    realized_perf_results.filter(period='reference', metrics=['recall']))
# ranker ranks on one drift method and one performance metric
correlation_ranked_features2 = ranker2.rank(
    univariate_results.filter(period='analysis', methods=['jensen_shannon']),
    realized_perf_results.filter(period='analysis', metrics=['recall']),
    only_drifting = False)

display(correlation_ranked_features2)

chunk                                                        \
           chunk                                                         
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                    debt_to_income_ratio                  \
                                      kolmogorov_smirnov                   
                 end_date    period                value upper_threshold   
0 2018-11-30 00:27:16.848  analysis              0.01576        0.018584   
1 2018-12-30 07:03:16.848  analysis              0.01268        0.018584   
2 2019-01-29 13:39:16.848  analysis              0.01734        0.018584   
3 2019-02-28 20:15:16.848  analysis              0.01280        0.018584   
4 2019-03-31 02:51:16.848  analysis              0.01918        0.018584   
5 2019-04-30 09:27:16.848  analysis              0.00824        0.018584   
6 2019-05-30 16:03:16.848  analysis              0.01058        0.018584   
7 2019-06-29 22:39:16.848  analysis              0.01002        0.018584   
8 2019-07-30 05:15:16.848  analysis              0.01068        0.018584   
9 2019-08-29 11:51:16.848  analysis              0.00680        0.018584   

                                                                                
                         jensen_shannon                                         
  lower_threshold  alert          value upper_threshold lower_threshold  alert  
0            None  False       0.031661        0.039328            None  False  
1            None  False       0.030011        0.039328            None  False  
2            None  False       0.031129        0.039328            None  False  
3            None  False       0.029464        0.039328            None  False  
4            None   True       0.030809        0.039328            None  False  
5            None  False       0.028681        0.039328            None  False  
6            None  False       0.043628        0.039328            None   True  
7            None  False       0.029253        0.039328            None  False  
8            None  False       0.030628        0.039328            None  False  
9            None  False       0.028330        0.039328            None  False

,number_of_alerts,column_name,rank
0,6,car_value,1
1,5,y_pred_proba,2
2,5,salary_range,3
3,5,repaid_loan_on_prev_car,4
4,5,loan_length,5
5,2,y_pred,6
6,2,repaid,7
7,1,debt_to_income_ratio,8
8,0,size_of_downpayment,9
9,0,driver_tenure,10


chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                      roc_auc                           ...  \
                 end_date    period     value sampling_error  realized  ...   
0 2018-11-30 00:27:16.848  analysis  0.970744       0.001814  0.970962  ...   
1 2018-12-30 07:03:16.848  analysis  0.971011       0.001814  0.970248  ...   
2 2019-01-29 13:39:16.848  analysis  0.971407       0.001814  0.976282  ...   
3 2019-02-28 20:15:16.848  analysis  0.971091       0.001814  0.967721  ...   
4 2019-03-31 02:51:16.848  analysis  0.971123       0.001814  0.969886  ...   
5 2019-04-30 09:27:16.848  analysis  0.961090       0.001814  0.960050  ...   
6 2019-05-30 16:03:16.848  analysis  0.961825       0.001814  0.958530  ...   
7 2019-06-29 22:39:16.848  analysis  0.961073       0.001814  0.959041  ...   
8 2019-07-30 05:15:16.848  analysis  0.962533       0.001814  0.963094  ...   
9 2019-08-29 11:51:16.848  analysis  0.961316       0.001814  0.957556  ...   

                            recall                           \
  lower_threshold  alert     value sampling_error  realized   
0        0.963317  False  0.928723       0.005137  0.930394   
1        0.963317  False  0.925261       0.005137  0.923922   
2        0.963317  False  0.929317       0.005137  0.938246   
3        0.963317  False  0.929713       0.005137  0.925060   
4        0.963317  False  0.930604       0.005137  0.927577   
5        0.963317   True  0.883990       0.005137  0.905086   
6        0.963317   True  0.883528       0.005137  0.899010   
7        0.963317   True  0.885501       0.005137  0.901718   
8        0.963317   True  0.885978       0.005137  0.906124   
9        0.963317   True  0.889808       0.005137  0.905823   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  0.944133                  0.913313        0.941033   
1                  0.940671                  0.909851        0.941033   
2                  0.944727                  0.913907        0.941033   
3                  0.945123                  0.914303        0.941033   
4                  0.946014                  0.915194        0.941033   
5                  0.899400                  0.868580        0.941033   
6                  0.898938                  0.868118        0.941033   
7                  0.900911                  0.870091        0.941033   
8                  0.901388                  0.870568        0.941033   
9                  0.905218                  0.874398        0.941033   

                          
  lower_threshold  alert  
0          0.9171  False  
1          0.9171  False  
2          0.9171  False  
3          0.9171  False  
4          0.9171  False  
5          0.9171   True  
6          0.9171   True  
7          0.9171   True  
8          0.9171   True  
9          0.9171   True  

[10 rows x 23 columns]

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                                                roc_auc  \
                 end_date    period targets_missing_rate sampling_error   
0 2018-11-30 00:27:16.848  analysis                  0.0       0.001814   
1 2018-12-30 07:03:16.848  analysis                  0.0       0.001814   
2 2019-01-29 13:39:16.848  analysis                  0.0       0.001814   
3 2019-02-28 20:15:16.848  analysis                  0.0       0.001814   
4 2019-03-31 02:51:16.848  analysis                  0.0       0.001814   
5 2019-04-30 09:27:16.848  analysis                  0.0       0.001814   
6 2019-05-30 16:03:16.848  analysis                  0.0       0.001814   
7 2019-06-29 22:39:16.848  analysis                  0.0       0.001814   
8 2019-07-30 05:15:16.848  analysis                  0.0       0.001814   
9 2019-08-29 11:51:16.848  analysis                  0.0       0.001814   

                                                           recall            \
      value upper_threshold lower_threshold  alert sampling_error     value   
0  0.970962         0.97866        0.963317  False       0.005137  0.930394   
1  0.970248         0.97866        0.963317  False       0.005137  0.923922   
2  0.976282         0.97866        0.963317  False       0.005137  0.938246   
3  0.967721         0.97866        0.963317  False       0.005137  0.925060   
4  0.969886         0.97866        0.963317  False       0.005137  0.927577   
5  0.960050         0.97866        0.963317   True       0.005137  0.905086   
6  0.958530         0.97866        0.963317   True       0.005137  0.899010   
7  0.959041         0.97866        0.963317   True       0.005137  0.901718   
8  0.963094         0.97866        0.963317   True       0.005137  0.906124   
9  0.957556         0.97866        0.963317   True       0.005137  0.905823   

                                          
  upper_threshold lower_threshold  alert  
0        0.941033          0.9171  False  
1        0.941033          0.9171  False  
2        0.941033          0.9171  False  
3        0.941033          0.9171  False  
4        0.941033          0.9171  False  
5        0.941033          0.9171   True  
6        0.941033          0.9171   True  
7        0.941033          0.9171   True  
8        0.941033          0.9171   True  
9        0.941033          0.9171   True

/home/jbris/.cache/pypoetry/virtualenvs/nannyml-test-mPICSMyd-py3.10/lib/python3.10/site-packages/nannyml/drift/ranker.py:376: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  pearsonr(filtered_values.ravel(), filtered_perf_change)


,column_name,pearsonr_correlation,pearsonr_pvalue,has_drifted,rank
0,repaid_loan_on_prev_car,0.998626,1.653702e-24,True,1
1,y_pred_proba,0.998586,2.141497e-24,True,2
2,salary_range,0.997379,5.487114e-22,True,3
3,loan_length,0.997314,6.833463e-22,True,4
4,car_value,0.997213,9.529448e-22,True,5
5,size_of_downpayment,0.311427,1.813554e-01,False,6
6,debt_to_income_ratio,0.256911,2.741989e-01,True,7
7,y_pred,0.066571,7.803558e-01,True,8
8,repaid,-0.127146,5.932180e-01,True,9
9,driver_tenure,-0.141105,5.529204e-01,False,10


/home/jbris/.cache/pypoetry/virtualenvs/nannyml-test-mPICSMyd-py3.10/lib/python3.10/site-packages/nannyml/drift/ranker.py:376: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  pearsonr(filtered_values.ravel(), filtered_perf_change)


,column_name,pearsonr_correlation,pearsonr_pvalue,has_drifted,rank
0,repaid_loan_on_prev_car,0.968970,0.000004,True,1
1,y_pred_proba,0.966157,0.000006,True,2
2,loan_length,0.965298,0.000006,True,3
3,car_value,0.963623,0.000007,True,4
4,salary_range,0.963456,0.000007,True,5
5,size_of_downpayment,0.308948,0.385072,False,6
6,debt_to_income_ratio,0.307373,0.387627,True,7
7,y_pred,-0.357571,0.310383,True,8
8,repaid,-0.395842,0.257495,True,9
9,driver_tenure,-0.575807,0.081520,False,10
